In [1]:
from correctionlib import schemav2 as cs
import uproot
import os

# Archivos por año
root_files = {
    "2022preEE": "../analysis/data/2022preEE_ScaleFactors_Muon_Z_HLT_abseta_pt.root",
    "2022postEE": "../analysis/data/2022postEE_ScaleFactors_Muon_Z_HLT_abseta_pt.root",
    "2023preBPix": "../analysis/data/2023preBPix_ScaleFactors_Muon_Z_HLT_abseta_pt.root",
    "2023postBPix": "../analysis/data/2023postBPix_ScaleFactors_Muon_Z_HLT_abseta_pt.root",
}

# Histogramas base
base_hists = {
    "Data": "NUM_IsoMu24_DEN_CutBasedIdTight_and_PFIsoTight_abseta_pt_efficiencyData",
    "MC":   "NUM_IsoMu24_DEN_CutBasedIdTight_and_PFIsoTight_abseta_pt_efficiencyMC"
}

# Inputs para Correction (sin 'year')
inputs = [
    cs.Variable(name="ValType", type="string", description="nominal/stat/syst"),
    cs.Variable(name="Path",    type="string", description="trigger path"),
    cs.Variable(name="eta",     type="real",   description="muon pseudorapidity"),
    cs.Variable(name="pt",      type="real",   description="muon pT [GeV]"),
]
output = cs.Variable(name="weight", type="real", description="efficiency or uncertainty")
valtypes = ["nominal", "stat", "syst"]
paths = ["NUM_IsoMu24_DEN_CutBasedIdTight_and_PFIsoTight"]

# Procesamiento por año
for year_tag, root_path in root_files.items():
    with uproot.open(root_path) as f:
        all_contents = {}
        edges_set = None

        for ds, base_hist in base_hists.items():
            try:
                th2_nom  = f[base_hist]
                th2_stat = f[f"{base_hist}_stat"]
                th2_syst = f[f"{base_hist}_combined_syst"]
            except KeyError as e:
                print(f"⚠️ Skipping {year_tag} {ds}: missing histogram - {e}")
                continue

            # Leer bin edges y asegurarse que están bien ordenados
            pt_edges  = th2_nom.axis(1).edges().tolist()  # X axis
            eta_edges = th2_nom.axis(0).edges().tolist()  # Y axis

            if not edges_set:
                print(f"✔️ Edges set for {year_tag}")
                print(f"    eta_edges: {eta_edges}")
                print(f"    pt_edges : {pt_edges}")
                edges_set = {"eta": eta_edges, "pt": pt_edges}

            # Transponer para que eta sea el eje exterior
            nom  = th2_nom.values().T.flatten(order="C").tolist()
            stat = th2_stat.values().T.flatten(order="C").tolist()
            syst = th2_syst.values().T.flatten(order="C").tolist()

            all_contents[ds] = {"nominal": nom, "stat": stat, "syst": syst}

        if not all_contents:
            print(f"⚠️ Skipping {year_tag}: no valid histograms.")
            continue

        corrections = []

        for ds, contents in all_contents.items():
            leaves = {}
            for vt in valtypes:
                content = contents[vt]
                n_eta = len(edges_set["eta"]) - 1
                n_pt  = len(edges_set["pt"]) - 1
                expected = n_eta * n_pt
                if len(content) != expected:
                    print(f"⚠️ Skipping {year_tag} {ds} {vt}: {len(content)} values, expected {expected}")
                    continue

                # ✅ Asegurar que inputs y edges están bien alineados
                mb = cs.MultiBinning(
                    nodetype="multibinning",
                    inputs=["eta", "pt"],
                    edges=[edges_set["eta"], edges_set["pt"]],
                    content=content,
                    flow="error"
                )
                leaves[vt] = mb

            # Nesting ValType → Path
            valtype_cat = cs.Category(
                nodetype="category",
                input="ValType",
                content=[
                    cs.CategoryItem(
                        key=vt,
                        value=cs.Category(
                            nodetype="category",
                            input="Path",
                            content=[
                                cs.CategoryItem(
                                    key=paths[0],
                                    value=leaves[vt]
                                )
                            ]
                        )
                    )
                    for vt in leaves
                ]
            )

            name = "Muon-HLT-DataEff" if ds == "Data" else "Muon-HLT-McEff"
            desc = f"Muon HLT {ds} efficiencies and uncertainties (nominal/stat/syst) for {year_tag}."

            corrections.append(
                cs.Correction(
                    name=name,
                    description=desc,
                    version=2,
                    inputs=inputs,
                    output=output,
                    data=valtype_cat
                )
            )

        cset = cs.CorrectionSet(
            schema_version=2,
            description=f"Muon HLT {year_tag} efficiencies and uncertainties",
            corrections=corrections
        )

        outname = f"../analysis/data/{year_tag}_Muon_HLT_Eff.json"
        with open(outname, "w") as fout:
            fout.write(cset.model_dump_json(indent=2, exclude_unset=True))
        print(f"✅ Written: {outname}")

✔️ Edges set for 2022preEE
    eta_edges: [0.0, 0.9, 1.2, 2.1, 2.4]
    pt_edges : [26.0, 30.0, 40.0, 50.0, 60.0, 120.0, 200.0, 500.0]
✅ Written: ../analysis/data/2022preEE_Muon_HLT_Eff.json
✔️ Edges set for 2022postEE
    eta_edges: [0.0, 0.9, 1.2, 2.1, 2.4]
    pt_edges : [26.0, 30.0, 40.0, 50.0, 60.0, 120.0, 200.0]
✅ Written: ../analysis/data/2022postEE_Muon_HLT_Eff.json
✔️ Edges set for 2023preBPix
    eta_edges: [0.0, 0.9, 1.2, 2.1, 2.4]
    pt_edges : [26.0, 30.0, 40.0, 50.0, 60.0, 120.0, 200.0]
✅ Written: ../analysis/data/2023preBPix_Muon_HLT_Eff.json
✔️ Edges set for 2023postBPix
    eta_edges: [0.0, 0.9, 1.2, 2.1, 2.4]
    pt_edges : [26.0, 30.0, 40.0, 50.0, 60.0, 120.0, 200.0]
✅ Written: ../analysis/data/2023postBPix_Muon_HLT_Eff.json
